In [192]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

分析10天上涨超过8的机会, 在何时出现
- sz_jt_10： 0,1,2,3： 0, 0-3，3-8, 8-...  
sql 
- "SELECT * from day_zhibiao where 
- jx_days_ud60 > 20 and jx_days_ud60 < 40 and \
- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 and \
- lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 "

In [193]:
sql = "SELECT * \
from ds_qf60"
print(sql)
df = pd.read_sql_query(sql, conn, index_col=None)

df.shape

SELECT * from ds_qf60


(35170, 47)

In [194]:
# df.info()

In [195]:
col_list_ds = ['symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'sz_jt_10','class0' ]
df.tail(1)[col_list_ds]

,symbol,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60,sz_max_5,sz_max_10,sz_max_20,sz_jt_10,class0
35169,689009,2021-04-19,3.04,-0.461137,-1.14476,1.36724,39,0.0,-0.143429,0.522034,...,0,1.30432,1.13952,0.745176,0.524615,0.379001,4.43571,4.43571,2.0,2


In [196]:
df['sz_jt_10'].value_counts(), df['sz_jt_10'].value_counts()/len(df)

(2.0    10830
 1.0    10013
 3.0     7440
 0.0     6887
 Name: sz_jt_10, dtype: int64,
 2.0    0.307933
 1.0    0.284703
 3.0    0.211544
 0.0    0.195820
 Name: sz_jt_10, dtype: float64)

In [197]:
# 开发测试：条件取数
# num = len(df)

# cond2a = df['sz_jt_10']>=2
# df[ cond2a ]['sz_jt_10'].value_counts()

# cond2b = df['sz_jt_10']>1 
# cond2c = df['sz_jt_10']<=2
# cond2bc2 = ( cond2b & cond2c )
# df[ cond2bc2 ]['sz_jt_10'].value_counts()

In [275]:
steps = 5
steps_dg_5, steps_dg_10 = 3,6
steps_fl_1_3, steps_fl_3_10  = 1, 1

In [276]:
# jx_dg_5 & 10 预处理
min_dg_10, max_dg_10 = df['jx_dg_10'].min(), df['jx_dg_10'].max()
step_dg_10 = (max_dg_10 - min_dg_10)/steps_dg_10

min_dg_5, max_dg_5 = df['jx_dg_5'].min(), df['jx_dg_5'].max()
step_dg_5 = (max_dg_5 - min_dg_5)/steps_dg_5


# fl_1_3 预处理
min_fl_1_3, max_fl_1_3 = df['pct_fl_1_3'].min(), df['pct_fl_1_3'].max()
step_fl_1_3 = (max_fl_1_3 - min_fl_1_3)/steps_fl_1_3



In [277]:
# epoch 初始化数据
# col = ['pct_change',
#     'diff', 'dea',\
#     'jx_days_ud60',\
#     'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
#     'jx_zs_5', 'jx_zs_10', 'jx_zs_20', \
#     'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
#     'sz_max_10' ]
col = ['jx_dg_5','pct_fl_1_3','step', 'class', 'rate' ]

def print_header():
    print('- jx_days_ud60 > 20 and jx_days_ud60 < 40 ')
    print('- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 ')
    print('- pct_fl_1_3 > 1.1579 and pct_fl_3_10>1.0730 ')
    print('-'*50)
    print('Loop      :%7s,%7s,%7s, %7s,%7s,%7s'%('10均fm','10线to','5均fm','5线to','1-3放','1-3量'),' = %5s,%6s,%5s'%('数量','达成','成功%'))


In [289]:
%%time
# man[col] = 0.0
man_data = {'jx_dg_5':0.0,
            'pct_fl_1_3':0.0,
            'step':0, 'class':0, 'rate':0.0    
}
man = pd.DataFrame(data=man_data, index=[0])
print_header()
all_step, all_class = 0, 0
# 开始epoch

cond_jt_10_3 = df['sz_jt_10']>=3
# cond_last = cond_begin

from_dg_10 = min_dg_10
for i_dg_10 in range(steps_dg_10-2):
    to_dg_10 = min_dg_10 + (i_dg_10+1)*step_dg_10
    # 生成条件
    cond_dg_10 = (df['jx_dg_10'] >= from_dg_10) & (df['jx_dg_10'] < to_dg_10 )
    # cond_last = (cond_dg_10) & ( cond_last )
    cond_last = (cond_dg_10) 

    from_dg_5 = min_dg_5
    for i_dg_5 in range(steps_dg_5):
        to_dg_5 = min_dg_5 + (i_dg_5+1)*step_dg_5
        # 生成条件
        cond_dg_5 = (df['jx_dg_5'] >= from_dg_5) & (df['jx_dg_5'] < to_dg_5 )
        cond_last = (cond_dg_5) & ( cond_last )
        
        from_fl_1_3 = min_fl_1_3
        for i_fl_1_3 in range(steps_fl_1_3):
            to_fl_1_3 = min_fl_1_3 + (i_fl_1_3+1)*step_fl_1_3
            # print('%.0f : , %.4f,  %.4f'%(i, from_fl_1_3, to_fl_1_3), end=' ' )
            # 生成条件
            cond_fl_1_3 = (df['pct_fl_1_3'] >= from_fl_1_3) & (df['pct_fl_1_3'] < to_fl_1_3 )
            cond_last = ( cond_fl_1_3 ) & (cond_last)
            cond_purpose = ( cond_last ) & (cond_jt_10_3)

            # 核心运算               
            num_step_series  = df[ cond_last ]['sz_jt_10'].value_counts() 
            num_class_series = df[ cond_purpose ]['sz_jt_10'].value_counts() 
            num_step = num_step_series.iloc[0] if not num_step_series.empty else 0
            num_class = num_class_series.iloc[0] if not num_class_series.empty else 0
            num_class_rate = 0 if num_class==0 else num_class*100/num_step
            print( '%2d %2d %2d '%(i_dg_10, i_dg_5, i_fl_1_3), \
                ':%8.4f,%8.4f,%8.4f,%8.4f,%8.4f,%8.4f'%(from_dg_10, to_dg_10, from_dg_5, to_dg_5, from_fl_1_3, to_fl_1_3),\
                ' = %7.0f,%7.0f,%7.2f'%(num_step, num_class, num_class_rate) )
            # 更新到man
            all_step  += num_step  ; all_class += num_class
            man_list = [from_dg_5, from_fl_1_3, num_step, num_class, num_class_rate]
            man = pd.DataFrame(np.insert(man.values, len(man.index), values=man_list, axis=0))        

            # 下一个 fl_1_3 loop
            from_fl_1_3 += step_fl_1_3
        # jx_dg_5下一个loop
        from_dg_5 += step_dg_5
    # jx_dg_5下一个loop
    from_dg_10 += step_dg_10
    
all_class_rate = all_class*100/all_step 
print( '-'*90)
print( ' '*65, '= %7.0f, %7.0f, %5.2f'%(all_step, all_class, all_class_rate) )
man.shape


- jx_days_ud60 > 20 and jx_days_ud60 < 40 
- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 
- pct_fl_1_3 > 1.1579 and pct_fl_3_10>1.0730 
--------------------------------------------------
Loop      :  10均fm,  10线to,   5均fm,    5线to,   1-3放,   1-3量  =    数量,    达成,  成功%
 0  0  0  :  0.0000,  0.4533, -1.8567,  0.8325,  1.1579, 66.3237  =    8341,   5054,  60.59
 0  1  0  :  0.0000,  0.4533,  0.8325,  3.5218,  1.1579, 66.3237  =       0,      0,   0.00
 0  2  0  :  0.0000,  0.4533,  3.5218,  6.2110,  1.1579, 66.3237  =       0,      0,   0.00
 1  0  0  :  0.4533,  0.9067, -1.8567,  0.8325,  1.1579, 66.3237  =     800,    642,  80.25
 1  1  0  :  0.4533,  0.9067,  0.8325,  3.5218,  1.1579, 66.3237  =       0,      0,   0.00
 1  2  0  :  0.4533,  0.9067,  3.5218,  6.2110,  1.1579, 66.3237  =       0,      0,   0.00
 2  0  0  :  0.9067,  1.3600, -1.8567,  0.8325,  1.1579, 66.3237  =      20,     20, 100.00
 2  1  0  :  0.9067,  1.3600,  0.8325,  3.5218,  1.1579, 66.3237  =       0,      0,   0

(13, 5)

In [279]:
# 数据核对 - 1
df[  (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()

3.0    7440
Name: sz_jt_10, dtype: int64

In [280]:
df[ (df['jx_dg_10']>=0) & (df['jx_dg_10']<0.4533) & (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()


3.0    5943
Name: sz_jt_10, dtype: int64

In [281]:
df[ (df['jx_dg_10']>=0.4533) & (df['jx_dg_10']<2.7200) & (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()

3.0    1497
Name: sz_jt_10, dtype: int64

In [282]:
df[ (df['jx_dg_10']>=0.4533) & (df['jx_dg_10']<2.7200) &\
    (df['jx_dg_5']>=-1.8567) & (df['jx_dg_5']<6.2110) &
    (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()


3.0    1497
Name: sz_jt_10, dtype: int64

In [283]:
df[ (df['jx_dg_10']>=0.4533) & (df['jx_dg_10']<2.7200) &\
    (df['jx_dg_5']>=-1.8567) & (df['jx_dg_5']<6.2110) &
    (df['pct_fl_1_3']>=1.1579) & (df['pct_fl_1_3']<66.3237) &
    (df['sz_jt_10']>=3) ]['sz_jt_10'].value_counts()

3.0    1496
Name: sz_jt_10, dtype: int64

In [284]:
pd.set_option('display.max_rows', None)
man.columns = col
# man.sort_index(ascending=True)
# man.set_index('rate', drop=False)
sorting = man.sort_values('rate', ascending=False)
sorting

,jx_dg_5,pct_fl_1_3,step,class,rate
7,-1.856730,1.15793,20.0,20.0,100.000000
10,-1.856730,1.15793,1.0,1.0,100.000000
4,-1.856730,1.15793,800.0,642.0,80.250000
1,-1.856730,1.15793,8341.0,5054.0,60.592255
0,0.000000,0.00000,0.0,0.0,0.000000
2,0.832523,1.15793,0.0,0.0,0.000000
3,3.521777,1.15793,0.0,0.0,0.000000
5,0.832523,1.15793,0.0,0.0,0.000000
6,3.521777,1.15793,0.0,0.0,0.000000
8,0.832523,1.15793,0.0,0.0,0.000000


In [285]:
cond_fl_1_3_class.value_counts()

False    35170
dtype: int64

In [286]:
# man